In [1]:
import pandas as pd
import numpy as np

import folium
from folium.plugins import AntPath

from pprint import pprint
from pprint import PrettyPrinter

In [2]:
df_intercambio = pd.read_csv('../Obtencion datos/intercambio_electrico.csv')

df_intercambio = df_intercambio.drop(columns=['Unnamed: 0'])

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-9297.7,0.166548,12/10/2024
1,Francia,Exportación,-8923.1,0.139081,13/10/2024
2,Francia,Exportación,-7027.8,0.127264,14/10/2024
3,Francia,Importación,46528.2,0.833452,12/10/2024
4,Francia,Importación,55234.3,0.860919,13/10/2024
5,Francia,Importación,48194.2,0.872736,14/10/2024
6,Francia,saldo,37230.5,1.000000,12/10/2024
7,Francia,saldo,46311.2,1.000000,13/10/2024
8,Francia,saldo,41166.4,1.000000,14/10/2024
9,Portugal,Exportación,-51614.9,0.745356,12/10/2024


In [ ]:
df_intercambio.columns

In [ ]:
df_intercambio['nombre'].unique()

In [ ]:
df_intercambio['tipo de intercambio'].unique()

In [34]:
def mapa_intercambio_energ(df):
  #Creamos mapa base:
  españa_alt = 40.4637
  españa_lat = -3.7492

  #Creacion del mapa

  from IPython.display import display

  spain_map = folium.Map(location = [españa_alt, españa_lat],
                       zoom_start= 5,
                       tiles = 'Esri Worldimagery',
                       width='500px',
                       height='500px'
                       )

  bounds =   [[51.1242, -17.0000],[20.0000, 9.6625]]

  spain_map.fit_bounds(bounds)


  #Coordenadas de los paises donde existe un intercambio energetico.
  lista_paises = ['Marruecos', 'Francia', 'Portugal', 'Andorra', 'España']
  lista_altitudes = [31.83999, 46.57771, 39.68023, 42.5462, 40.4637]
  lista_latitudes = [-6.19721, 2.78159, -8.80606, 1.5034, -3.7492]

  df_coordenadas = pd.DataFrame()
  df_coordenadas['nombre'] = lista_paises
  df_coordenadas['altitud'] = lista_altitudes
  df_coordenadas['latitud'] = lista_latitudes


  #Creacion de variables con sus coordenadas
  marruecos = [df_coordenadas['altitud'][0], df_coordenadas['latitud'][0]]
  francia = [df_coordenadas['altitud'][1], df_coordenadas['latitud'][1]]
  portugal = [df_coordenadas['altitud'][2], df_coordenadas['latitud'][2]]
  andorra = [df_coordenadas['altitud'][3], df_coordenadas['latitud'][3]]
  españa = [df_coordenadas['altitud'][4], df_coordenadas['latitud'][4]]


  #Unión de df_coordenadas y df incluido en la función - en nuestro caso, df_intercambio.
  df_unido = pd.merge(df_intercambio, df_coordenadas, how = 'left', on = 'nombre')

  #Creación de emojis por país

  #Si no está instalado, pip install emoji
  import emoji

  esp_emoji = emoji.emojize(':Spain:')
  mar_emoji = emoji.emojize(":Morocco:")
  fra_emoji = emoji.emojize(':France:')
  por_emoji = emoji.emojize(':Portugal:')
  and_emoji = emoji.emojize(':Andorra:')


#CREACIÓN DEL MAPA CON INFORMACIÓN
  intercambios = folium.map.FeatureGroup(name='Intercambios')
  ##IMPUT DEL TIPO DE INFORMACIÓN DESEADA.
  input_tipo_inter = input('Introduce el tipo de intercambio: importación, exportación, o saldo: ______ ').lower()
  ## df_filtrado es el df resultante del tipo de intercambio deseado.
  df_filtrado = df_unido[df_unido['tipo de intercambio'].str.lower() == input_tipo_inter]


  for lat, lng, pais, valores, porcentaje, fecha in zip(df_filtrado['altitud'],
                           df_filtrado['latitud'],
                           df_filtrado['nombre'],
                           df_filtrado['Valores'],
                           df_filtrado['Porcentaje'],
                           df_filtrado['Fecha actualización']):

        contenido_label = f'''<b> Pais: {pais} </b><br>
                            <b>Tipo de intercambio: {input_tipo_inter} </b><br>
                            <b>Valores: {valores} </b><br>
                            <b>Porcentaje: {porcentaje} </b><br>
                            <b>Fecha actualización: {fecha} </b>'''
        intercambios.add_child(folium.Marker(location=[lat, lng],
                                        popup=contenido_label))



  spain_map.add_child(intercambios)



  folium.Marker(
          location= españa,
          icon=folium.DivIcon(
                  html = f'<div style="font-size: 50px;" >{esp_emoji}</div>'),
          popup = 'España'
          ).add_to(spain_map)

  folium.Marker(
          location= portugal,
          icon=folium.DivIcon(
                  html = f'<div style="font-size: 30px;" >{por_emoji}</div>'),
          popup = 'Portugal'
          ).add_to(spain_map)
  folium.Marker(
          location= francia,
          icon=folium.DivIcon(
                  html = f'<div style="font-size: 30px;" >{fra_emoji}</div>'),
          popup = 'Francia'
          ).add_to(spain_map)
  folium.Marker(
          location= marruecos,
          icon=folium.DivIcon(
                  html = f'<div style="font-size: 30px;" >{mar_emoji}</div>'),
          popup = 'Marruecos'
          ).add_to(spain_map)
  folium.Marker(
          location= andorra,
          icon=folium.DivIcon(
                  html = f'<div style="font-size: 30px;" >{and_emoji}</div>'),
          popup = 'Andorra'
          ).add_to(spain_map)




  for i, v in df_filtrado[df_filtrado['nombre'] == 'Francia'].iterrows():
        if v['Valores'] < 0:
                AntPath(locations = [españa, francia],
                color = 'blue',
                delay = 2000,
                weight = 5).add_to(spain_map)
        elif v['Valores'] > 0:
                AntPath(locations = [francia,españa],
                color = 'blue',
                delay = 2000,
                weight = 5).add_to(spain_map)
        else:
                pass

  for i, v in df_filtrado[df_filtrado['nombre'] == 'Andorra'].iterrows():
        if v['Valores'] < 0:
                AntPath(locations = [españa, andorra],
                color = 'yellow',
                delay = 2000,
                weight = 5).add_to(spain_map)
        elif v['Valores'] > 0:
                AntPath(locations = [andorra,españa],
                color = 'yellow',
                delay = 2000,
                weight = 5).add_to(spain_map)
        else:
                pass

  for i, v in df_filtrado[df_filtrado['nombre'] == 'Marruecos'].iterrows():
        if v['Valores'] < 0:
                AntPath(locations = [españa, marruecos],
                color = 'red',
                delay = 2000,
                weight = 5).add_to(spain_map)
        elif v['Valores'] > 0:
                AntPath(locations = [marruecos, españa],
                color = 'red',
                delay = 2000,
                weight = 5).add_to(spain_map)
        else:
                pass

  for i, v in df_filtrado[df_filtrado['nombre'] == 'Portugal'].iterrows():
        if v['Valores'] < 0:
                AntPath(locations = [españa, portugal],
                color = 'green',
                delay = 2000,
                weight = 5).add_to(spain_map)
        elif v['Valores'] > 0:
                AntPath(locations = [portugal, españa],
                color = 'green',
                delay = 2000,
                weight = 5).add_to(spain_map)
        else:
                pass



  return spain_map









In [35]:
mapa_intercambio_energ(df_intercambio)